# ⚙️ Pipeline ETL: Customer Churn
**Objetivo:** Extrair os dados brutos, realizar a limpeza e padronização, salvar um checkpoint local e carregar os dados no PostgreSQL.

In [ ]:
# IMPORTAÇÕES E CONFIGURAÇÕES DE AMBIENTE
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv, find_dotenv
from pathlib import Path

caminho_env = find_dotenv()
load_dotenv(caminho_env)
pasta_raiz = Path(caminho_env).parent

caminho_csv = pasta_raiz / 'data' / 'raw' / 'WA_Fn-UseC_-Telco-Customer-Churn.csv'
caminho_processed = pasta_raiz / 'data' / 'processed' / 'cleaned_telco_churn.csv'

DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

print("✅ Ambiente configurado com sucesso!")

---
### 1. Extração
Leitura do arquivo CSV original da pasta `raw`.

In [ ]:
try:
    df_clientes = pd.read_csv(caminho_csv)
    print("✅ Passo 1: Dados originais lidos com sucesso!")
except FileNotFoundError:
    print(f"❌ Erro: Arquivo não encontrado no caminho: {caminho_csv}")
    raise

---
### 2. Transformação 
Padronização do nome das colunas (snake_case), tipagem de dados financeiros e tratamento de valores nulos.

In [ ]:
df_clientes.columns = [
    'customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
    'tenure', 'phone_service', 'multiple_lines', 'internet_service',
    'online_security', 'online_backup', 'device_protection', 'tech_support',
    'streaming_tv', 'streaming_movies', 'contract', 'paperless_billing',
    'payment_method', 'monthly_charges', 'total_charges', 'churn'
]

df_clientes['total_charges'] = pd.to_numeric(df_clientes['total_charges'], errors='coerce')
df_clientes['total_charges'] = df_clientes['total_charges'].fillna(0)

print("✅ Passo 2: Dados limpos e padronizados!")

---
### 3. Checkpoint Físico (Save Processed)
Salvamento dos dados limpos em um novo arquivo CSV. Isso garante que usuários sem acesso ao Banco de Dados possam consumir os dados tratados (Opção B do README).

In [ ]:
# Salvando o arquivo localmente ANTES de tentar conectar no banco
df_clientes.to_csv(caminho_processed, index=False)
print(f"✅ Passo 3: Checkpoint salvo! Arquivo físico gerado em: {caminho_processed}")

---
### 4. Carga de Dados (Load)
Conexão com o PostgreSQL e injeção dos dados na tabela `raw_customers`.

In [ ]:
print("⏳ Passo 4: Conectando e enviando dados para o PostgreSQL...")

db_url = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(db_url)

try:
    # O if_exists='append' injeta os dados respeitando a nossa estrutura DDL
    df_clientes.to_sql('raw_customers', engine, if_exists='append', index=False)
    print("🚀 SUCESSO! Banco de dados populado com os dados limpos.")
except Exception as erro:
    print(f"❌ Erro ao inserir no banco: {erro}")
    print("💡 Dica: Verifique se a tabela 'raw_customers' foi criada via DBeaver e se os dados já não foram inseridos antes.")